In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA


In [2]:
df = pd.read_csv("global_disaster_response_2018_2024 (1).csv")
df.head()


,date,country,disaster_type,severity_index,casualties,economic_loss_usd,response_time_hours,aid_amount_usd,response_efficiency_score,recovery_days,latitude,longitude
0,2021-01-31,Brazil,Earthquake,5.99,111,7934365.71,15.62,271603.79,83.21,67,-30.613,-122.557
1,2018-12-23,Brazil,Extreme Heat,6.53,100,8307648.99,5.03,265873.81,96.18,55,10.859,-159.194
2,2020-08-10,India,Hurricane,1.55,22,765136.99,32.54,49356.49,60.40,22,0.643,-160.978
3,2022-09-15,Indonesia,Extreme Heat,4.55,94,1308251.31,7.83,237512.88,86.41,47,-33.547,30.350
4,2022-09-28,United States,Wildfire,3.80,64,2655864.36,21.90,188910.69,72.81,42,-19.170,-117.137


In [3]:
df = df.copy()

# Drop date
df.drop(columns=['date'], inplace=True)

# Label Encoding
le_country = LabelEncoder()
le_disaster = LabelEncoder()

df['country'] = le_country.fit_transform(df['country'])
df['disaster_type'] = le_disaster.fit_transform(df['disaster_type'])

# Create binary target
df['severity_high'] = (df['severity_index'] > 5).astype(int)

# Define features + target
X = df.drop(columns=['severity_high'])
y = df['severity_high']

X.head()


,country,disaster_type,severity_index,casualties,economic_loss_usd,response_time_hours,aid_amount_usd,response_efficiency_score,recovery_days,latitude,longitude
0,2,1,5.99,111,7934365.71,15.62,271603.79,83.21,67,-30.613,-122.557
1,2,2,6.53,100,8307648.99,5.03,265873.81,96.18,55,10.859,-159.194
2,9,4,1.55,22,765136.99,32.54,49356.49,60.40,22,0.643,-160.978
3,10,2,4.55,94,1308251.31,7.83,237512.88,86.41,47,-33.547,30.350
4,19,9,3.80,64,2655864.36,21.90,188910.69,72.81,42,-19.170,-117.137


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
params = {
    "C": [0.1, 1, 10],
    "gamma": ["scale", "auto", 0.1, 0.01],
    "kernel": ["rbf"]
}

grid = GridSearchCV(SVC(), params, cv=5)
grid.fit(X_train_scaled, y_train)

print("Best Parameters:", grid.best_params_)
print("Best Score:", grid.best_score_)
scores = cross_val_score(SVC(kernel='rbf', C=1), X, y, cv=5)
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())


In [7]:
linear_svm = SVC(kernel='linear')
linear_svm.fit(X_train_scaled, y_train)

y_pred_linear = linear_svm.predict(X_test_scaled)

print("Linear SVM Accuracy:", accuracy_score(y_test, y_pred_linear))
print(classification_report(y_test, y_pred_linear))


Linear SVM Accuracy: 0.9983
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5066
           1       1.00      1.00      1.00      4934

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(StandardScaler().fit_transform(X))

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42
)

model_2d = SVC(kernel='rbf', C=1)
model_2d.fit(X_train2, y_train2)

# Meshgrid
x_min, x_max = X_reduced[:,0].min()-1, X_reduced[:,0].max()+1
y_min, y_max = X_reduced[:,1].min()-1, X_reduced[:,1].max()+1

xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 200),
    np.linspace(y_min, y_max, 200)
)

Z = model_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(9,7))
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_reduced[:,0], X_reduced[:,1], c=y, edgecolor='k')
plt.title("SVM Decision Boundary (PCA 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

